In [1]:
import pandas as pd
import numpy as np

df1 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\CENTRO_Lazio.csv", sep=";")
df2 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\CENTRO_Marche.csv", sep=";")
df3 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\CENTRO_Toscana.csv", sep=";")
df4 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\CENTRO_Umbria.csv", sep=";")

df_CENTRO = pd.concat([df1, df2, df3, df4])

df1 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\Isole_Sardegna.csv", sep=";")
df2 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\Isole_Sicilia.csv", sep=";")

df_Isole = pd.concat([df1, df2])

df1 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\NORD_EST_Bolzano_Bozen.csv", sep=";")
df2 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\NORD_EST_Emilia_Romagna.csv", sep=";")
df3 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\NORD_EST_FINAL_3.csv", sep=";")
df4 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\NORD_EST_Friuli_Venezia_Giulia.csv", sep=";")

df_NORD_EST = pd.concat([df1, df2, df3, df4])

df1 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\NORD_OVEST_Liguria.csv", sep=";")
df2 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\NORD_OVEST_Lombardia.csv", sep=";")
df3 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\NORD_OVEST_Piemonte.csv", sep=";")
df4 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\NORD_OVEST_Valle_D_Aosta.csv", sep=";")

df_NORD_OVEST = pd.concat([df1, df2, df3, df4])

df1 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\Sud_Abruzzo_Basilicata_Calabria.csv", sep=";")
df2 = pd.read_csv(r"C:\Users\AdamMorgan\OneDrive\UCL\Research Project\Data\Climate data\SUD_Campania_Molise_Puglia.csv", sep=";")

df_SUD = pd.concat([df1, df2])


In [2]:
# Join tables together
df = pd.concat([df_CENTRO, df_Isole, df_NORD_EST, df_NORD_OVEST, df_SUD])

In [3]:
# Convert string to Date Time
df['date'] = pd.to_datetime(df['DAY'], format='%Y%m%d')

# Extract year, month, and day to new columns
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [4]:
# Convert the 'date' column to datetime format if it isn't already
df['date'] = pd.to_datetime(df['date'])

# Define a function to determine the crop year
def assign_crop_year(date):
    if (date >= pd.Timestamp('2011-12-01')) and (date <= pd.Timestamp('2012-05-31')):
        return 2012
    if (date >= pd.Timestamp('2012-12-01')) and (date <= pd.Timestamp('2013-05-31')):
        return 2013
    elif (date >= pd.Timestamp('2013-12-01')) and (date <= pd.Timestamp('2014-05-31')):
        return 2014
    else:
        return None  # or any other default value for dates outside the provided ranges

#df = df.drop(columns=['year'])
# Apply the function to each date in the 'date' column to assign the crop year
df['crop_year'] = df['date'].apply(assign_crop_year)

df.head(3)

,GRID_NO,LATITUDE,LONGITUDE,ALTITUDE,DAY,TEMPERATURE_AVG,PRECIPITATION,ET0,date,year,month,day,crop_year
0,59119,41.39576,12.93197,11,19790101,12.3,0.6,NaN,1979-01-01,1979,1,1,NaN
1,59119,41.39576,12.93197,11,19790102,4.6,0.1,NaN,1979-01-02,1979,1,2,NaN
2,59119,41.39576,12.93197,11,19790103,-0.8,0.5,NaN,1979-01-03,1979,1,3,NaN


In [5]:
df = df.dropna(subset=['crop_year'])
df['crop_year'] = df['crop_year'].astype(int)
df = df.drop(columns=['ET0'])

In [6]:
# Groupby and aggregate
avg_df = df.groupby(['GRID_NO', 'LATITUDE', 'LONGITUDE', 'year', 'crop_year', 'month']).agg({
    'PRECIPITATION':'sum', 
    'TEMPERATURE_AVG': ['mean', 'min', 'max']
}).reset_index()

# Flatten multi-level columns
avg_df.columns = [f'{col[0]}_{col[1]}' if col[1] else col[0] for col in avg_df.columns.values]

# Rename columns for clarity
avg_df.rename(columns={
    'TEMPERATURE_AVG_mean': 'avg_temperature',
    'TEMPERATURE_AVG_min': 'min_temp',
    'TEMPERATURE_AVG_max': 'max_temp',
    'PRECIPITATION_sum': 'total_monthly_precipitation'
}, inplace=True)

df = avg_df
df.head(3)

,GRID_NO,LATITUDE,LONGITUDE,year,crop_year,month,total_monthly_precipitation,avg_temperature,min_temp,max_temp
0,33119,35.51234,12.58303,2011,2012,12,69.9,15.896774,11.5,19.0
1,33119,35.51234,12.58303,2012,2012,1,69.5,13.683871,12.5,15.5
2,33119,35.51234,12.58303,2012,2012,2,57.4,11.741379,8.5,15.0


In [7]:
# Dictionary mapping names to GRID_NO values
grid_name_mapping = {
    69120: 'AGUGLIANO',
    74114: 'ARGELATO',
    43123: 'CAMMARATA',
    75109: 'FIORENZUOLA',
    60128: 'FOGGIA',
    43126: 'LIBERTINIA',
    63118: 'MONTELIBRETTI',
    76107: 'S.ANGELO LODIGIANO',
    42122: 'S.STEFANO QUISQUINIA',
    68120: 'TOLENTINO'
}

# Filter the original DataFrame for the specified GRID_NO values
filtered_df = df[df['GRID_NO'].isin(grid_name_mapping.keys())].copy()

# Add the respective names for each GRID_NO to the filtered DataFrame
filtered_df['Name'] = filtered_df['GRID_NO'].map(grid_name_mapping)
filtered_df.head(5)

,GRID_NO,LATITUDE,LONGITUDE,year,crop_year,month,total_monthly_precipitation,avg_temperature,min_temp,max_temp,Name
396,42122,37.44685,13.55037,2011,2012,12,46.2,11.693548,6.5,19.0,S.STEFANO QUISQUINIA
397,42122,37.44685,13.55037,2012,2012,1,54.2,9.122581,6.4,12.9,S.STEFANO QUISQUINIA
398,42122,37.44685,13.55037,2012,2012,2,93.3,8.148276,4.5,12.6,S.STEFANO QUISQUINIA
399,42122,37.44685,13.55037,2012,2012,3,43.7,11.900000,6.0,16.0,S.STEFANO QUISQUINIA
400,42122,37.44685,13.55037,2012,2012,4,36.6,16.000000,10.8,21.5,S.STEFANO QUISQUINIA


In [8]:
# Reorder columns
column_order = ['GRID_NO', 'Name', 'LONGITUDE', 'LATITUDE', 'crop_year', 'year', 'month', 'total_monthly_precipitation', 'avg_temperature', 'max_temp', 'min_temp']
df = filtered_df[column_order]
df.head(3)

,GRID_NO,Name,LONGITUDE,LATITUDE,crop_year,year,month,total_monthly_precipitation,avg_temperature,max_temp,min_temp
396,42122,S.STEFANO QUISQUINIA,13.55037,37.44685,2012,2011,12,46.2,11.693548,19.0,6.5
397,42122,S.STEFANO QUISQUINIA,13.55037,37.44685,2012,2012,1,54.2,9.122581,12.9,6.4
398,42122,S.STEFANO QUISQUINIA,13.55037,37.44685,2012,2012,2,93.3,8.148276,12.6,4.5


In [9]:
df = df.rename(columns={
    'GRID_NO': 'MARS ID',
    'Name': 'Location',
    'LATITUDE': 'coordinate x',
    'LONGITUDE': 'coordinate y'
})

In [10]:
df12 = df[df['crop_year'] == 2012].reset_index(drop=True)
df13 = df[df['crop_year'] == 2013].reset_index(drop=True)
df14 = df[df['crop_year'] == 2014].reset_index(drop=True)

In [11]:
def transform_dataframe(df):
    def format_year_month_with_prefix(month, prefix):
        month_str_mapping = {
            1: 'JAN', 2: 'FEB', 3: 'MAR', 4: 'APR', 5: 'MAY', 6: 'JUN',
            7: 'JUL', 8: 'AUG', 9: 'SEP', 10: 'OCT', 11: 'NOV', 12: 'DEC'
        }
        month_str = month_str_mapping[month]
        formatted_str = prefix + "_" + month_str
        return formatted_str

    data_dicts = []

    grouped = df.groupby(['Location', 'crop_year'])

    for (location, year), group_df in grouped:
        data_dict = {
            'Location': location,
            'crop_year': year
        }
        for _, row in group_df.iterrows():
            precip_col = format_year_month_with_prefix(row['month'], 'TMP')
            temp_col = format_year_month_with_prefix(row['month'], 'AMT')
            max_col = format_year_month_with_prefix(row['month'], 'MaxT')
            min_col = format_year_month_with_prefix(row['month'], 'MinT')
            
            data_dict[precip_col] = row['total_monthly_precipitation']
            data_dict[temp_col] = row['avg_temperature']
            data_dict[max_col] = row['max_temp']
            data_dict[min_col] = row['min_temp']
        
        data_dicts.append(data_dict)

    # Convert list of dictionaries into a DataFrame
    result_df = pd.DataFrame(data_dicts)

    # Ensuring the location is a categorical column for correct sorting
    result_df['Location'] = result_df['Location'].astype('category')

    return result_df

# Now, you can transform df12, df13, and df14 by calling:
result_df12 = transform_dataframe(df12)
result_df13 = transform_dataframe(df13)
result_df14 = transform_dataframe(df14)

In [12]:
combined_df = pd.concat([result_df12, result_df13, result_df14], ignore_index=True)
combined_df.head(5)

,Location,crop_year,TMP_DEC,AMT_DEC,MaxT_DEC,MinT_DEC,TMP_JAN,AMT_JAN,MaxT_JAN,MinT_JAN,...,MaxT_MAR,MinT_MAR,TMP_APR,AMT_APR,MaxT_APR,MinT_APR,TMP_MAY,AMT_MAY,MaxT_MAY,MinT_MAY
0,AGUGLIANO,2012,46.4,8.258065,14.8,3.5,20.7,4.919355,10.8,0.2,...,18.7,5.2,43.1,13.166667,18.4,7.2,32.9,15.841935,19.6,11.2
1,ARGELATO,2012,11.4,4.838710,8.1,-0.4,4.7,2.609677,6.7,-2.0,...,18.8,6.9,66.8,13.756667,20.3,7.5,61.2,18.564516,24.2,14.0
2,CAMMARATA,2012,42.8,9.629032,12.9,4.7,58.6,7.429032,10.5,4.8,...,14.6,4.8,38.4,13.673333,19.2,9.3,11.8,16.912903,20.8,13.5
3,FIORENZUOLA,2012,25.2,3.851613,8.8,-0.3,25.8,2.545161,7.0,-2.8,...,20.3,6.2,206.2,12.473333,18.8,6.5,145.2,17.280645,22.9,12.4
4,FOGGIA,2012,22.9,10.635484,16.7,3.7,19.0,7.900000,12.1,2.4,...,17.5,7.9,48.2,14.823333,21.2,8.5,16.4,18.377419,21.7,13.1


In [13]:
combined_df.to_csv('temp_rainfall.csv', index=False)